<a href="https://colab.research.google.com/github/datascientist-ld1981/Speech-Recognition/blob/main/Evolution_of_speech_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install fastdtw
!pip install sounddevice wavio
!pip install pydub
!apt-get install -y ffmpeg
!pip install pydub numpy librosa fastdtw


📌 Program 1: DTW-Based Speech Recognition (Oldest Model)
•	Model Type: Template Matching
•	Key Concept: Dynamic Time Warping (DTW) for aligning speech signals
•	Stages Covered:
1.	Analog to Digital Conversion (ADC)
2.	Preprocessing (Noise Reduction, VAD)
3.	Feature Extraction (MFCCs, Spectrograms)
4.	Phoneme Recognition (Pattern Matching)
5.	Lexical Modeling (Template Matching)
6.	Speech-to-Text (DTW for Alignment)
7.	NLP for Understanding (Basic Word Matching)


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 29 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 29 not upgraded.


📌 Program 1: DTW-Based Speech Recognition (Oldest Model)
•	Model Type: Template Matching
•	Key Concept: Dynamic Time Warping (DTW) for aligning speech signals
•	Stages Covered:
1.	Analog to Digital Conversion (ADC)
2.	Preprocessing (Noise Reduction, VAD)
3.	Feature Extraction (MFCCs, Spectrograms)
4.	Phoneme Recognition (Pattern Matching)
5.	Lexical Modeling (Template Matching)
6.	Speech-to-Text (DTW for Alignment)
7.	NLP for Understanding (Basic Word Matching)


In [13]:
import numpy as np
import librosa
import librosa.display
import matplotlib.pyplot as plt
from scipy.spatial.distance import cdist
from fastdtw import fastdtw
from pydub import AudioSegment
from pydub.playback import play
import io

# Step 1: Convert Audio File to WAV Format
def convert_to_wav(input_file, output_file):
    audio = AudioSegment.from_file(input_file)
    audio.export(output_file, format="wav")
    return output_file

# Step 2: Extract MFCC Features
def extract_mfcc(audio_file):
    y, sr = librosa.load(audio_file, sr=16000)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
    return mfcc.T

# Step 3: DTW Alignment Function
def dtw_distance(mfcc1, mfcc2):
    distance, path = fastdtw(mfcc1, mfcc2, dist=lambda x, y: np.linalg.norm(x - y, ord=1))
    return distance

# Step 4: Recognize Speech from Template Database
def recognize_speech(test_mfcc, templates):
    min_distance = float('inf')
    best_match = None

    for word, mfcc in templates.items():
        distance = dtw_distance(test_mfcc, mfcc)
        print(f"Distance to {word}: {distance}")

        if distance < min_distance:
            min_distance = distance
            best_match = word

    return best_match

# Step 5: Main Function (Convert, Extract, and Recognize)
def main():
    templates = {
        'hello': extract_mfcc('hello.wav'),
        #'world': extract_mfcc('world.wav')
    }

    #convert_to_wav("test_input.mp3", "test.wav")
    test_mfcc = extract_mfcc("hello.wav")
    recognized_word = recognize_speech(test_mfcc, templates)
    print("Recognized Word:", recognized_word)

if __name__ == "__main__":
    main()


<ipython-input-13-5ef0e4e278ad>:19: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(audio_file, sr=16000)
/usr/local/lib/python3.11/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)
<ipython-input-13-5ef0e4e278ad>:19: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(audio_file, sr=16000)
/usr/local/lib/python3.11/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


Distance to hello: 0.0
Recognized Word: hello


🔹 Program 2: HMM-Based Speech Recognition
This program improves on DTW by using a Hidden Markov Model (HMM) for better speech recognition.
✅ Features:
✔ Uses Gaussian HMMs to model speech sequences
✔ Extracts MFCC features
✔ Trains the model using sample words
✔ Recognizes the spoken word


In [39]:
import numpy as np
import librosa
import librosa.display
import matplotlib.pyplot as plt
from scipy.spatial.distance import cdist
from fastdtw import fastdtw
import os

# Step 1: Extract MFCC Features
def extract_mfcc(y, sr):
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
    return mfcc.T

# Step 2: Store Training Phrases
def train_dtw(audio_file, phrases):
    if not os.path.exists(audio_file):
        print(f"Error: {audio_file} not found.")
        return None

    y, sr = librosa.load(audio_file, sr=16000)
    mfcc = extract_mfcc(y, sr)

    trained_models = {}
    for phrase, start_end in phrases.items():
        start, end = start_end
        trained_models[phrase] = mfcc[start:end]

    return trained_models

# Step 3: Recognize Speech using DTW
def recognize_speech(audio_file, trained_models):
    if trained_models is None:
        return "No model trained."

    y, sr = librosa.load(audio_file, sr=16000)
    test_mfcc = extract_mfcc(y, sr)

    best_match = None
    best_distance = float('inf')

    for phrase, mfcc_template in trained_models.items():
        distance, _ = fastdtw(test_mfcc, mfcc_template, dist=lambda x, y: np.linalg.norm(x - y))
        if distance < best_distance:
            best_distance = distance
            best_match = phrase

    print("Recognition Distance:", best_distance)
    return best_match if best_match else "No match found"

# Step 4: Main Function
def main():
    audio_file = 'hello.wav'  # Single file containing all speech
    phrases = {
        "hello": (0, 20),
        "how are you": (21, 50),
        "good morning everyone": (51, 90),
        "have a nice day": (91, 130)
    }

    trained_models = train_dtw(audio_file, phrases)
    result = recognize_speech(audio_file, trained_models)
    print("Recognition Result:", result)

if __name__ == "__main__":
    main()


<ipython-input-39-df48002e4a2d>:20: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(audio_file, sr=16000)
/usr/local/lib/python3.11/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)
<ipython-input-39-df48002e4a2d>:35: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(audio_file, sr=16000)
/usr/local/lib/python3.11/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


Recognition Distance: 14797.133999401873
Recognition Result: good morning everyone


In [17]:
!pip install hmmlearn
!pip install --upgrade hmmlearn


Program 3
•  Load Pretrained Model & Processor
•	Import Wav2Vec2Processor and Wav2Vec2ForCTC.
•	Load facebook/wav2vec2-base-960h model.
•  Load and Preprocess Audio File
•	Use librosa to load hello.wav at 16 kHz sampling rate.
•	Convert raw waveform into tensor format for model processing.
•  Perform Speech Recognition
•	Pass the processed audio into the Wav2Vec2 model.
•	Compute logits (predicted probabilities of speech units).
•  Decode Predictions
•	Find the most likely predicted tokens using argmax().
•	Convert token IDs into text transcription using batch_decode().
•  Display Recognized Speech
•	Print the transcription result from hello.wav.





In [43]:
import torch
import torchaudio
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC
import librosa

# Load Pretrained Wav2Vec2 Model and Processor
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")

def recognize_speech(audio_file):
    # Load audio file
    y, sr = librosa.load(audio_file, sr=16000)
    input_values = processor(y, return_tensors="pt", sampling_rate=16000).input_values

    # Perform inference
    with torch.no_grad():
        logits = model(input_values).logits

    # Decode prediction
    predicted_ids = torch.argmax(logits, dim=-1)
    transcription = processor.batch_decode(predicted_ids)[0]

    return transcription

# Main Function
def main():
    audio_file = 'hello.wav'  # Single file containing all speech
    result = recognize_speech(audio_file)
    print("Recognition Result:", result)

if __name__ == "__main__":
    main()


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/159 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/163 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.60k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/291 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/378M [00:00<?, ?B/s]

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-43-aa384de09004>:12: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(audio_file, sr=16000)
/usr/local/lib/python3.11/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


Recognition Result: HALLO HOW ARE YOU GOOD MORNING A RIVER HOW A MACE D


In [45]:
import torch
import torchaudio
import librosa
import numpy as np
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC

# Load Pretrained Wav2Vec2 Model and Processor
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")

def load_audio(audio_file):
    waveform, sample_rate = torchaudio.load(audio_file)
    return waveform, sample_rate

# Recognize Speech using Wav2Vec2
def recognize_speech(audio_file):
    waveform, sample_rate = load_audio(audio_file)
    if sample_rate != 16000:
        transform = torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=16000)
        waveform = transform(waveform)

    input_values = processor(waveform.squeeze().numpy(), return_tensors="pt", sampling_rate=16000).input_values
    with torch.no_grad():
        logits = model(input_values).logits

    predicted_ids = torch.argmax(logits, dim=-1)
    transcription = processor.batch_decode(predicted_ids)[0]
    return transcription

# Main Function
def main():
    audio_file = 'hello.wav'
    result = recognize_speech(audio_file)
    print("Recognition Result:", result)

if __name__ == "__main__":
    main()


Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Recognition Result: HALLO HOW ARE YOU GOOD MORNING E RIVER HOW E MACE D


Program 4
Upgrade to OpenAI Whisper
🔹 Better transcription quality
🔹 Handles accents, background noise, and unclear speech
🔹 Supports multiple languages

In [1]:
!pip install openai-whisper


In [2]:
import torch
import torchaudio
import whisper

def load_audio(audio_file):
    waveform, sample_rate = torchaudio.load(audio_file)
    return waveform, sample_rate

# Load Whisper Model
model = whisper.load_model("base")

def recognize_speech(audio_file):
    result = model.transcribe(audio_file)
    return result["text"]

# Main Function
def main():
    audio_file = 'hello.wav'
    result = recognize_speech(audio_file)
    print("Recognition Result:", result)

if __name__ == "__main__":
    main()


/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Recognition Result:  Hello how are you? Good morning everyone have a nice day
